Importing Data and packages

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import params as params

base_der = pd.read_csv('data/case1/base-case-DER.csv', skiprows=1, usecols=[0, 1, 2, 3], header=None)
base_load = pd.read_csv('data/case1/base-case-load.csv', skiprows=1, usecols=[0, 1, 2, 3, 4], header=None)
base_line = pd.read_csv('data/case1/base-case-line.csv', skiprows=1, usecols=[0, 1, 2, 3, 4, 5], header=None)

DERs = set()
CLs = set()

for u in base_der[0]:
    DERs.add(u)

for load, precedance in zip(base_load[0], base_load[1]):
    if precedance == 'Priority':
        CLs.add(load)

Adding edges and weights to graph

In [ ]:
G = nx.Graph()
wt = {}
for u, v, w in zip(base_line[0], base_line[1], base_line[3]):
    G.add_edge(u, v)
    wt[(u, v)] = w
    wt[(v, u)] = w

Finding all simple paths from each critical load to each source

In [ ]:
path_list = []
for cl in CLs:
    tmp = []
    for path in nx.all_simple_paths(G, source=cl, target=DERs):
        loads, sources = 0, 0
        for node in path:
            if node in CLs:
                loads += 1
            if node in DERs:
                sources += 1
        if loads > 1 or sources > 1:
            continue
        tmp.append(list(nx.utils.pairwise(path)))
    if len(tmp) == 0: continue
    path_list.append(tmp)
print(path_list)


All possible path combination calculation

In [ ]:
import itertools

FNs = []
raw_combs = []
for element in itertools.product(*path_list):
    tmp = []
    for path in element:
        for edge in path:
            tmp.append(edge)
    raw_combs.append(list(element))
    FNs.append(tmp)


In [ ]:
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, font_color='w', font_size=10)
nx.draw_networkx_nodes(G, pos, list(DERs), node_color='g')
nx.draw_networkx_nodes(G, pos, list(CLs), node_color='r')
nx.draw_networkx_edges(G, pos, [k for k in wt.keys() if wt[k] == 0], style=':', edge_color='w')
nx.draw_networkx_edges(G, pos, FNs[2], width=4)
plt.show()

### Calculating parameters to measure resiliency of possible network

Calculation of Parameter matrix

In [ ]:
rsl = params.ratio_source_load(FNs, CLs, DERs)
ops = params.switch_ops(FNs, wt)
cen = params.agg_centrality(FNs)
obs = params.overlapping_branches(FNs)
apl = params.avg_path_length(raw_combs)

df = pd.DataFrame(np.array([rsl, ops, cen, obs, apl]), columns=[f"FN{i}" for i in range(1, len(rsl)+1)])
df.index = ['rsl', 'ops', 'cen', 'obs', 'apl']
df = df.T
print(df)

Calculation of weighted mean and resiliency

In [ ]:
df['wtd_mean'] = (df['rsl'] * 6 + df['ops'] * 3 + df['cen'] * 1 + df['obs'] * 5 + df['apl'] * 5)/20
df['resiliency'] = np.exp(-df['wtd_mean'])
print(df.sort_values('resiliency', ascending=False))

Finding critical lines based on network topology